# Level Pool Reservoir Routing in DDR

This notebook demonstrates DDR's **level pool reservoir routing** module — a physics-based
approach that integrates reservoir outflow directly into the Muskingum-Cunge sparse solve
via an **RHS override**, replacing MC routing at reservoir reaches with weir + orifice
outflow controlled by pool elevation.

**What you'll see:**
1. Outflow rating curve (orifice-only vs weir+orifice regimes)
2. Equilibrium pool initialization (orifice inversion)
3. Synthetic flood event routed through a real HydroLAKES reservoir
4. Peak attenuation + timing lag in hydrographs
5. Mass balance verification

The functions `_level_pool_outflow` and `_compute_equilibrium_pool_elevation` are imported
directly from `ddr.routing.mmc` — the same code used during DDR training and inference.

**How it works in the sparse solve:**
- `b[res] = _level_pool_outflow(pool_elevation)` — override RHS with reservoir outflow
- `c_1_[res] = 0` — identity row in matrix, so solve gives `Q[res] = b[res] = outflow`
- Pool elevation updated via forward Euler after the solve

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

# Ensure DDR package is importable
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root != project_root.parent:
    project_root = project_root.parent
src_path = str(project_root / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from ddr.routing.mmc import _compute_equilibrium_pool_elevation, _level_pool_outflow

# Load reservoir parameters from the HydroLAKES-MERIT intersection CSV
reservoir_csv = project_root / "data" / "merit_reservoir_params.csv"
df = pd.read_csv(reservoir_csv, index_col="COMID")
print(f"Loaded {len(df)} reservoir reaches")
print("\nSummary statistics:")
print(df[["lake_area_m2", "weir_elevation", "orifice_elevation", "weir_length", "orifice_area"]].describe())

# Pick a representative mid-sized reservoir (near median lake area)
COMID = 74004852
res = df.loc[COMID]
print(f"\nSelected COMID {COMID}:")
print(res.to_string())

## Physics: Level Pool Equations

The level pool method assumes a **horizontal water surface** in the reservoir.
Outflow depends only on pool elevation $H$ via two structures:

### Orifice discharge (low-level outlet)
$$Q_o = C_o \cdot A_o \cdot \sqrt{2g(H - H_o)} \quad \text{when } H > H_o$$

### Weir discharge (spillway)
$$Q_w = C_w \cdot L_w \cdot (H - H_w)^{3/2} \quad \text{when } H > H_w$$

### Pool elevation update (forward Euler, after sparse solve)
$$H^{t+1} = H^t + \frac{\Delta t}{A_{lake}} \cdot (Q_{in} - Q_{out})$$

where $Q_{in} = (N \cdot Q^{t+1})_{res} + q'_{res}$ uses the solved upstream discharge.

### Equilibrium pool initialization (orifice inversion)
$$h_{eq} = \frac{Q_{hotstart}^2}{2g \cdot (C_o \cdot A_o)^2}, \quad H_{init} = \min(H_o + h_{eq},\; H_w)$$

```
                    Weir crest (H_w)
                    vvvvvvvvvvvvv
   ~~~~~~~~~~~~~~~~/             \~~~~~~~~~~~~~~~~  <-- Pool elevation (H)
   |              |               |              |
   |              |   Reservoir   |              |
   |              |               |              |
   |              |    [  O  ] ---|-- Orifice    |  <-- Orifice elevation (H_o)
   |              |               |              |
   +--------------+---------------+--------------+
        Inflow -->                  --> Outflow
```

| Symbol | Description | Units |
|--------|-------------|-------|
| $C_o$ | Orifice coefficient | ~ 0.6 |
| $A_o$ | Orifice area | m$^2$ |
| $C_w$ | Weir coefficient | ~ 0.4 |
| $L_w$ | Weir length | m |
| $A_{lake}$ | Lake surface area | m$^2$ |

In [ ]:
# --- Outflow Rating Curve ---
# Sweep pool elevation from below orifice to above weir and show the two regimes.

orifice_elev = res["orifice_elevation"]
weir_elev = res["weir_elevation"]
initial_elev = res["initial_pool_elevation"]

# Sweep range: from 2m below orifice to 3m above weir
elev_min = orifice_elev - 2.0
elev_max = weir_elev + 3.0
elevations = torch.linspace(elev_min, elev_max, 500)

discharge_lb = torch.tensor(0.0001)
outflows = _level_pool_outflow(
    pool_elevation=elevations,
    weir_elevation=torch.tensor(weir_elev),
    orifice_elevation=torch.tensor(orifice_elev),
    weir_coeff=torch.tensor(res["weir_coeff"]),
    weir_length=torch.tensor(res["weir_length"]),
    orifice_coeff=torch.tensor(res["orifice_coeff"]),
    orifice_area=torch.tensor(res["orifice_area"]),
    discharge_lb=discharge_lb,
)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(elevations.numpy(), outflows.numpy(), "k-", linewidth=2)

# Annotate zones
ax.axvline(orifice_elev, color="C0", linestyle="--", alpha=0.7, label=f"Orifice elev = {orifice_elev:.1f} m")
ax.axvline(weir_elev, color="C3", linestyle="--", alpha=0.7, label=f"Weir elev = {weir_elev:.1f} m")
ax.axvline(initial_elev, color="C2", linestyle=":", alpha=0.7, label=f"Initial pool = {initial_elev:.1f} m")

# Zone labels
mid_no_flow = (elev_min + orifice_elev) / 2
mid_orifice = (orifice_elev + weir_elev) / 2
mid_both = (weir_elev + elev_max) / 2
y_top = outflows.max().item() * 0.9
ax.text(mid_no_flow, y_top * 0.1, "No flow", ha="center", fontsize=10, color="gray")
ax.text(mid_orifice, y_top * 0.3, "Orifice only", ha="center", fontsize=10, color="C0")
ax.text(mid_both, y_top * 0.7, "Weir + Orifice", ha="center", fontsize=10, color="C3")

ax.set_xlabel("Pool Elevation [m]")
ax.set_ylabel("Outflow [m$^3$/s]")
ax.set_title(f"Outflow Rating Curve — COMID {COMID}")
ax.legend(loc="upper left")
ax.set_xlim(elev_min, elev_max)
ax.set_ylim(bottom=0)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# --- Equilibrium Pool Initialization ---
# Instead of using a fixed initial_pool_elevation from HydroLAKES (which can be
# inconsistent with flow conditions), DDR computes the equilibrium pool elevation
# where orifice outflow matches the hotstart inflow.

# Simulate a range of inflows and show the equilibrium pool elevation
test_inflows = torch.linspace(0.1, 50.0, 200)
orifice_elev = torch.tensor(res["orifice_elevation"])
orifice_coeff = torch.tensor(res["orifice_coeff"])
orifice_area_t = torch.tensor(res["orifice_area"])
weir_elev_t = torch.tensor(res["weir_elevation"])

pool_eq = _compute_equilibrium_pool_elevation(
    inflow=test_inflows,
    orifice_elevation=orifice_elev.expand_as(test_inflows),
    orifice_coeff=orifice_coeff.expand_as(test_inflows),
    orifice_area=orifice_area_t.expand_as(test_inflows),
    weir_elevation=weir_elev_t.expand_as(test_inflows),
)

# Verify: outflow at equilibrium ≈ inflow (for sub-weir cases)
outflow_at_eq = _level_pool_outflow(
    pool_elevation=pool_eq,
    weir_elevation=weir_elev_t.expand_as(test_inflows),
    orifice_elevation=orifice_elev.expand_as(test_inflows),
    weir_coeff=torch.tensor(res["weir_coeff"]).expand_as(test_inflows),
    weir_length=torch.tensor(res["weir_length"]).expand_as(test_inflows),
    orifice_coeff=orifice_coeff.expand_as(test_inflows),
    orifice_area=orifice_area_t.expand_as(test_inflows),
    discharge_lb=discharge_lb,
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Left: equilibrium pool elevation vs inflow
ax1.plot(test_inflows.numpy(), pool_eq.numpy(), "k-", linewidth=2)
ax1.axhline(
    weir_elev_t.item(), color="C3", linestyle="--", alpha=0.7, label=f"Weir = {weir_elev_t.item():.1f} m"
)
ax1.axhline(
    orifice_elev.item(), color="C0", linestyle="--", alpha=0.7, label=f"Orifice = {orifice_elev.item():.1f} m"
)
csv_init = res["initial_pool_elevation"]
ax1.axhline(csv_init, color="C2", linestyle=":", alpha=0.7, label=f"CSV init = {csv_init:.1f} m")
ax1.set_xlabel("Inflow [m$^3$/s]")
ax1.set_ylabel("Equilibrium Pool Elevation [m]")
ax1.set_title("Equilibrium Pool Initialization")
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Right: outflow at equilibrium vs inflow (should be 1:1 below cap)
ax2.plot(test_inflows.numpy(), outflow_at_eq.numpy(), "C0-", linewidth=2, label="Outflow at equilibrium")
ax2.plot(test_inflows.numpy(), test_inflows.numpy(), "k--", alpha=0.5, label="1:1 line")
ax2.set_xlabel("Inflow [m$^3$/s]")
ax2.set_ylabel("Outflow [m$^3$/s]")
ax2.set_title("Outflow = Inflow at Equilibrium")
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# For our selected reservoir, what would the equilibrium be?
sample_inflow = torch.tensor(5.0)  # typical baseflow
eq_elev = _compute_equilibrium_pool_elevation(
    inflow=sample_inflow.unsqueeze(0),
    orifice_elevation=orifice_elev.unsqueeze(0),
    orifice_coeff=orifice_coeff.unsqueeze(0),
    orifice_area=orifice_area_t.unsqueeze(0),
    weir_elevation=weir_elev_t.unsqueeze(0),
)
print(f"CSV initial_pool_elevation: {csv_init:.2f} m")
print(f"Equilibrium pool at Q=5 m³/s: {eq_elev.item():.2f} m")
print(f"Difference: {csv_init - eq_elev.item():.2f} m (CSV is higher → would cause excess outflow)")

In [ ]:
# --- Simulate a Synthetic Flood Event ---
# Triangular flood pulse (48h rise, 96h fall) on top of baseflow.
# Uses _level_pool_outflow + inline forward Euler (same math as route_timestep).

dt = 3600.0  # 1-hour timestep [s] (same as DDR routing)
n_hours = 336  # 14 days
hours = np.arange(n_hours)

# Synthetic inflow: baseflow + triangular flood pulse
baseflow = 5.0  # m^3/s
peak_flow = 80.0  # m^3/s
rise_hours = 48
fall_hours = 96

inflow = np.full(n_hours, baseflow)
# Rising limb (hours 24-72)
rise_start = 24
for h in range(rise_hours):
    t = rise_start + h
    if t < n_hours:
        inflow[t] = baseflow + (peak_flow - baseflow) * (h / rise_hours)
# Falling limb (hours 72-168)
fall_start = rise_start + rise_hours
for h in range(fall_hours):
    t = fall_start + h
    if t < n_hours:
        inflow[t] = peak_flow - (peak_flow - baseflow) * (h / fall_hours)

inflow_tensor = torch.tensor(inflow, dtype=torch.float64)

# Reservoir parameters as tensors
lake_area_m2 = torch.tensor(res["lake_area_m2"], dtype=torch.float64)
weir_elevation = torch.tensor(res["weir_elevation"], dtype=torch.float64)
orifice_elevation = torch.tensor(res["orifice_elevation"], dtype=torch.float64)
weir_coeff = torch.tensor(res["weir_coeff"], dtype=torch.float64)
weir_length = torch.tensor(res["weir_length"], dtype=torch.float64)
orifice_coeff = torch.tensor(res["orifice_coeff"], dtype=torch.float64)
orifice_area = torch.tensor(res["orifice_area"], dtype=torch.float64)
discharge_lb_64 = torch.tensor(0.0001, dtype=torch.float64)

# Outflow kwargs (shared across all timesteps)
outflow_kw = {  # type: ignore
    "weir_elevation": weir_elevation,
    "orifice_elevation": orifice_elevation,
    "weir_coeff": weir_coeff,
    "weir_length": weir_length,
    "orifice_coeff": orifice_coeff,
    "orifice_area": orifice_area,
    "discharge_lb": discharge_lb_64,
}

# Initialize pool at equilibrium (same as DDR training)
pool_elev = _compute_equilibrium_pool_elevation(
    inflow=torch.tensor(baseflow, dtype=torch.float64).unsqueeze(0),
    orifice_elevation=orifice_elevation.unsqueeze(0),
    orifice_coeff=orifice_coeff.unsqueeze(0),
    orifice_area=orifice_area.unsqueeze(0),
    weir_elevation=weir_elevation.unsqueeze(0),
).squeeze(0)
print(f"Equilibrium init: {pool_elev.item():.2f} m (vs CSV: {res['initial_pool_elevation']:.2f} m)")

# Run level pool routing: outflow + forward Euler pool update
outflow_series = np.zeros(n_hours)
pool_elev_series = np.zeros(n_hours)

for t in range(n_hours):
    outflow_t = _level_pool_outflow(pool_elevation=pool_elev, **outflow_kw)
    dh = dt * (inflow_tensor[t] - outflow_t) / (lake_area_m2 + 1e-8)
    pool_elev = pool_elev + dh
    outflow_series[t] = outflow_t.item()
    pool_elev_series[t] = pool_elev.item()

print(f"Inflow peak:  {inflow.max():.1f} m^3/s at hour {inflow.argmax()}")
print(f"Outflow peak: {outflow_series.max():.1f} m^3/s at hour {outflow_series.argmax()}")
print(f"Peak reduction: {(1 - outflow_series.max() / inflow.max()) * 100:.1f}%")
print(f"Timing lag: {outflow_series.argmax() - inflow.argmax()} hours")

In [ ]:
# --- Hydrograph Plot (Main Figure) ---
# Top: inflow vs outflow.  Bottom: pool elevation with reference lines.

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), sharex=True, gridspec_kw={"height_ratios": [2, 1]})

# Top panel: Hydrographs
ax1.fill_between(hours, inflow, alpha=0.15, color="C0")
ax1.plot(hours, inflow, "C0-", linewidth=1.5, label="Inflow")
ax1.plot(hours, outflow_series, "C3-", linewidth=2, label="Outflow")

# Annotate peak reduction
peak_in_h = int(inflow.argmax())
peak_out_h = int(outflow_series.argmax())
ax1.annotate(
    f"Peak reduction: {(1 - outflow_series.max() / inflow.max()) * 100:.0f}%",
    xy=(peak_out_h, outflow_series.max()),
    xytext=(peak_out_h + 30, outflow_series.max() + 10),
    arrowprops={"arrowstyle": "->", "color": "C3"},  # type: ignore
    fontsize=10,
    color="C3",
)
ax1.annotate(
    f"Lag: {peak_out_h - peak_in_h}h",
    xy=((peak_in_h + peak_out_h) / 2, inflow.max() * 0.5),
    fontsize=10,
    ha="center",
    color="gray",
)

ax1.set_ylabel("Discharge [m$^3$/s]")
ax1.set_title(
    f"Level Pool Reservoir Routing — COMID {COMID} (lake area = {res['lake_area_m2'] / 1e6:.1f} km$^2$)"
)
ax1.legend(loc="upper right")
ax1.grid(True, alpha=0.3)

# Bottom panel: Pool elevation
ax2.plot(hours, pool_elev_series, "k-", linewidth=1.5, label="Pool elevation")
ax2.axhline(weir_elev, color="C3", linestyle="--", alpha=0.7, label=f"Weir = {weir_elev:.1f} m")
ax2.axhline(orifice_elev, color="C0", linestyle="--", alpha=0.7, label=f"Orifice = {orifice_elev:.1f} m")

ax2.set_xlabel("Time [hours]")
ax2.set_ylabel("Elevation [m]")
ax2.legend(loc="upper right", fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- Mass Balance Verification ---
# Cumulative inflow should equal cumulative outflow + change in storage.

cumulative_inflow = np.cumsum(inflow) * dt  # m^3
cumulative_outflow = np.cumsum(outflow_series) * dt  # m^3

# Storage change: delta_H * lake_area (relative to equilibrium init, not CSV)
initial_pool = pool_elev_series[0] - (
    dt * (inflow[0] - outflow_series[0]) / (res["lake_area_m2"] + 1e-8)
)  # back out the pre-first-step elevation
storage_change = (pool_elev_series - initial_pool) * res["lake_area_m2"]  # m^3

# Mass balance: inflow = outflow + storage_change
mass_balance_error = cumulative_inflow - cumulative_outflow - storage_change
relative_error = mass_balance_error[-1] / cumulative_inflow[-1]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Top: cumulative volumes
ax1.plot(hours, cumulative_inflow / 1e6, "C0-", linewidth=1.5, label="Cumulative inflow")
ax1.plot(
    hours,
    (cumulative_outflow + storage_change) / 1e6,
    "C3--",
    linewidth=1.5,
    label="Cumulative outflow + $\\Delta$storage",
)
ax1.set_ylabel("Volume [10$^6$ m$^3$]")
ax1.set_title("Mass Balance Verification")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bottom: residual error
ax2.plot(hours, mass_balance_error, "k-", linewidth=1)
ax2.axhline(0, color="gray", linestyle="-", alpha=0.3)
ax2.set_xlabel("Time [hours]")
ax2.set_ylabel("Mass balance error [m$^3$]")
ax2.set_title(f"Relative error = {relative_error:.2e}")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final cumulative inflow:  {cumulative_inflow[-1]:.2f} m^3")
print(f"Final cumulative outflow: {cumulative_outflow[-1]:.2f} m^3")
print(f"Storage change:           {storage_change[-1]:.2f} m^3")
print(f"Mass balance error:       {mass_balance_error[-1]:.6f} m^3")
print(f"Relative error:           {relative_error:.2e}")
assert abs(relative_error) < 1e-5, f"Mass balance violation: {relative_error:.2e}"